<a href="https://colab.research.google.com/github/Balachandar-Ganesan/ExamplesforLearningQuestDB/blob/main/GCPCaseStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-cloud-pubsub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 6.7 MB/s eta 0:00:00


In [3]:
import requests
import json
import time
from google.colab import auth
from google.cloud import pubsub_v1


API_KEY = "TG4P32WYA41OZJR8"
STOCKS = ['AAPL','MSFT','NVDA','AMZN','GOOGL']
API_URL = "https://www.alphavantage.co/query"
PROJECT_ID = "fifth-battery-341710"
TOPIC_ID = "stock-data-topic"

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(PROJECT_ID, TOPIC_ID)

def publish_to_pubsub(data):
    payload = json.dumps(data).encode("utf-8")
    future = publisher.publish(topic_path, payload)
    print(f"Published message ID: {future.result()}")


auth.authenticate_user(project_id=PROJECT_ID)
for stock in STOCKS:
  time.sleep(3)
  params = {
    "function": "GLOBAL_QUOTE",
    "symbol": stock,
    "apikey": API_KEY
  }


  response = requests.get(API_URL, params=params)
  if response.status_code == 200:
    data = response.json()
    print(f"--- Live Global Quote for {stock} ---")
    print(json.dumps(data, indent=4))
    publish_to_pubsub({
            "symbol": stock,
            "data": data
        })
  else:
    print(f"Request failed with status code: {response.status_code}")

--- Live Global Quote for GOOGL ---
{
    "Global Quote": {
        "01. symbol": "GOOGL",
        "02. open": "334.9500",
        "03. high": "340.4900",
        "04. low": "333.6200",
        "05. price": "335.9700",
        "06. volume": "33517607",
        "07. latest trading day": "2026-01-13",
        "08. previous close": "331.8600",
        "09. change": "4.1100",
        "10. change percent": "1.2385%"
    }
}
Published message ID: 17821063254326930


SELECT
data['symbol'],
 LAX_STRING(data['data']['Global Quote']['01. symbol']) AS quote_symbol,
    LAX_FLOAT64(data['data']['Global Quote']['02. open']) AS open_price,
LAX_FLOAT64(data['data']['Global Quote']['03. high']) AS high_price,
    LAX_FLOAT64(data['data']['Global Quote']['04. low']) AS low_price,
    LAX_FLOAT64(data['data']['Global Quote']['05. price']) AS current_price,
    LAX_INT64(data['data']['Global Quote']['06. volume']) AS volume,
    PARSE_DATE('%Y-%m-%d', LAX_STRING(data['data']['Global Quote']['07. latest trading day'])) AS latest_trading_day,
    LAX_FLOAT64(data['data']['Global Quote']['08. previous close']) AS previous_close
 FROM `fifth-battery-341710.StockPrices.DailyStockPrices` LIMIT 1000

SELECT TO_JSON_STRING(data, true) from `StockPrices.DailyStockPrices`;
SELECT JSON_VALUE(data, "$.symbol") AS symbol from `StockPrices.DailyStockPrices`;
SELECT JSON_QUERY(data, "$.data.Global Quote") AS OpenPrice from `StockPrices.DailyStockPrices`;



service-PROJECT_NUMBER@gcp-sa-pubsub.iam.gserviceaccount.com